# Set up MongoDB

Let's first get set up with some prerequisites that you will need to successfully execute the project.

In this project, you will build a text-to-query agent that can run queries against a movies database stored in MongoDB. But first, you will need a MongoDB Atlas account with a database cluster. **Follow these steps to get set up**:    
* **Register for a [free MongoDB Atlas account](https://www.mongodb.com/cloud/atlas/register/?utm_campaign=devrel&utm_source=third-party-content&utm_medium=cta&utm_content=datacamp_text_to_query_course&utm_term=apoorva.joshi)**
* **Create a new database cluster**: Once you register and sign into your Atlas account for the first time, you will be directed to the Cluster Deployment page.
  * Select the _Free_ option to create a free tier cluster.
  Notice that this will automatically check the _Preload sample dataset_ box under _Quick setup_. This will load a movies database named `sample_mflix` into your cluster once it is created. We will use this database throughout the project.
  * Click _Create Deployment_ to create the cluster.
  * In the modal that appears, click _Create database user_. Then click _Choose a connection method_.
  * In the next screen, click _Drivers_.
  * Next, copy the connection string (starts with `mongodb+srv://`) to a safe place. You will need this to connect to your cluster later on.
  * Click _Done_.
* **Allow Access from anywhere**: To connect to your MongoDB cluster from your Datalab workbook, you will need to open up network access to your cluster.
  * From the side navigation bar in the Atlas UI, select _Security_ > Network Access.
  * On the screen that appears, click _Add IP Address_.
  * In the modal that appears, click _Allow Access From Anywhere_ and click _Confirm_.
  NOTE: Opening access to your MongoDB clusters from anywhere is not recommended in production environments. We are just doing it for easy access in this project.

In [ ]:
# !pip install  --quiet pymongo==4.13.2

In [5]:
import os
from pymongo import MongoClient

# provide your own MONGODB_URI and OPENAI_API_KEY
MONGODB_URI = "write_your_own_mongodb_uri"
OPENAI_API_KEY = "write_your_own_openai_api_key"

# Initialize a MongoDB Python client
mongodb_client = MongoClient(MONGODB_URI)

In [3]:
# Check the connection to your cluster
ping_result = mongodb_client.admin.command("ping")
print(ping_result)

{'ok': 1}


# Writing MongoDB queries

The MongoDB Query API is the mechanism with which you (or AI agents) interact with data stored in MongoDB.

In the following section, we will perform some queries against the sample movies dataset using the MongoDB Query API.

The movies dataset is actually a database consisting of multiple collections that you can write queries against.

Let's first analyze what collections are available in this database.

In `pymongo`, MongoDB's Python driver, you can access a database using dictionary keys.

In [4]:
# Access the sample_mflix database
db = mongodb_client["sample_mflix"]
db.list_collection_names()

['embedded_movies', 'movies', 'sessions', 'comments', 'users', 'theaters']

Looking at the output of the above command, the `sample_mflix` database has the following collections:

* `comments`: Contains comments associated with specific movies.
* `movies`: Contains movie information, including release year, director, and reviews.
* `embedded_movies`: Contains details about a subset of movies in the `movies` collection, with additional embedding fields added to the documents to facilitate vector search.
* `users`: Contains user information.
* `theaters`: Contains locations of movie theaters.
* `sessions`: Contains comments associated with specific movies.

For now, let's pick the `movies` collection and perform some queries against it.

### Basic CRUD operations

The simplest way to interact with your MongoDB data is via CRUD (Create, Read, Update, and Delete) operations.

Let's try out some of the CRUD operations using MongoDB's Python driver (`pymongo`).

To preview a random sample document in the `movies` collection, we will call the `find_one()` method from the Python driver with an empty query filter (`{}`).


In [7]:
collection = db["movies"]
collection.find_one({})

{'_id': ObjectId('573a1390f29313caabcd42e8'),
 'plot': 'A group of bandits stage a brazen train hold-up, only to find a determined posse hot on their heels.',
 'genres': ['Short', 'Western'],
 'runtime': 11,
 'cast': ['A.C. Abadie',
  "Gilbert M. 'Broncho Billy' Anderson",
  'George Barnes',
  'Justus D. Barnes'],
 'poster': 'https://m.media-amazon.com/images/M/MV5BMTU3NjE5NzYtYTYyNS00MDVmLWIwYjgtMmYwYWIxZDYyNzU2XkEyXkFqcGdeQXVyNzQzNzQxNzI@._V1_SY1000_SX677_AL_.jpg',
 'title': 'The Great Train Robbery',
 'fullplot': "Among the earliest existing films in American cinema - notable as the first film that presented a narrative story to tell - it depicts a group of cowboy outlaws who hold up a train and rob the passengers. They are then pursued by a Sheriff's posse. Several scenes have color included - all hand tinted.",
 'languages': ['English'],
 'released': datetime.datetime(1903, 12, 1, 0, 0),
 'directors': ['Edwin S. Porter'],
 'rated': 'TV-G',
 'awards': {'wins': 1, 'nominations': 0, 

Let's try a query with a simple query filter.

To find all movies that were released before the year 1900, we will call the `find()` method with a query filter.

The filter is essentially a Python dictionary where the key is the field in the MongoDB documents that you want to filter on (`year`), and the value is the filter expression, which uses the `$lt` operator to convey `< 1900`.

Note that the `find()` method returns a cursor that you need to iterate over, to access the returned documents.

In [11]:
old_movies_query = {"year": {"$lt": 1900}}

import pprint

# Execute the query and iterate through the resulting cursor
for doc in collection.find(old_movies_query):
    pprint.pprint(doc)

{'_id': ObjectId('573a139cf29313caabcf560f'),
 'awards': {'nominations': 0, 'text': '1 win.', 'wins': 1},
 'cast': ['May Irwin', 'John C. Rice'],
 'countries': ['USA'],
 'directors': ['William Heise'],
 'fullplot': 'A couple make love, in the 19th century sense. Then he grooms '
             'his moustache, and gives her a kiss. On the lips!',
 'genres': ['Short', 'Romance'],
 'imdb': {'id': 139738, 'rating': 5.9, 'votes': 1739},
 'lastupdated': '2015-08-07 00:29:36.180000000',
 'num_mflix_comments': 0,
 'plot': 'Two people kiss.',
 'runtime': 1,
 'title': 'The Kiss',
 'tomatoes': {'dvd': datetime.datetime(1999, 5, 4, 0, 0),
              'lastUpdated': datetime.datetime(2015, 8, 22, 19, 24, 9),
              'viewer': {'meter': 87, 'numReviews': 235, 'rating': 4.0}},
 'type': 'movie',
 'writers': ['John J. McNally (play)'],
 'year': 1896}
{'_id': ObjectId('573a13a0f29313caabd041db'),
 'awards': {'nominations': 0, 'text': '1 win.', 'wins': 1},
 'cast': ['May Irwin', 'John C. Rice'],
 '

Now, let's try a query with multiple conditions in the query filter. For this, we will use the `$and` operator, similar to the `AND` operator in SQL.

The `$and` operator accepts a list of query expressions, so in this case we have one expression to convey `year` < 1930 using the `$lt` operator, and another for `imdb.rating` >= 8 using the `$gte` operator.

In [13]:
best_old_movies_query = { "$and":[
                    {"year": {"$lt": 1921}},
                    {"imdb.rating": {"$gte": 8}}
                  ]
        }

# Execute the query and iterate through the resulting cursor
for doc in collection.find(best_old_movies_query):
    pprint.pprint(doc)

{'_id': ObjectId('573a1391f29313caabcd6e2a'),
 'awards': {'nominations': 0, 'text': '1 win.', 'wins': 1},
 'cast': ['Buster Keaton', 'Sybil Seely'],
 'countries': ['USA'],
 'directors': ['Edward F. Cline', 'Buster Keaton'],
 'fullplot': 'Buster and Sybil exit a chapel as newlyweds. Among the gifts is '
             "a portable house you easily put together in one week. It doesn't "
             "help that Buster's rival for Sybil switches the numbers on the "
             'crates containing the house parts.',
 'genres': ['Short', 'Comedy'],
 'imdb': {'id': 11541, 'rating': 8.3, 'votes': 3942},
 'languages': ['English'],
 'lastupdated': '2015-05-07 01:07:01.633000000',
 'num_mflix_comments': 0,
 'plot': 'A newly wedded couple attempt to build a house with a prefabricated '
         "kit, unaware that a rival sabotaged the kit's component numbering.",
 'rated': 'TV-G',
 'released': datetime.datetime(1920, 9, 1, 0, 0),
 'runtime': 25,
 'title': 'One Week',
 'tomatoes': {'lastUpdated': dat

# More complex aggregations

You can also perform complex aggregations on your data, involving grouping and joins, using the MongoDB Query API.

As you have seen, this is done using aggregation pipelines, which sequentially process documents using a series of data processing "stages".

Let's run some aggregation pipelines against the `movies` collection.

### Top 5 movies with the highest IMDB ratings

To get the top 5 movies with the highest IMDB ratings, you need to:
1. Sort the documents in descending order of `imdb.rating`.
2. Limit the results to 5.
3. Project out only the `title` field in the final results.

Each of the steps above corresponds to a stage in the aggregation pipeline. As with CRUD operations, you will use different operators available in the MongoDB Query API to build the aggregation pipeline. For example, you will use `$sort` to sort the results in a particular order, `$limit` to limit the number of results returned, and `$project` to project out only certain fields.

**Create the aggregation pipeline to list the top 5 movies with highest IMDB ratings consisting of `$sort`, `$limit` and `$project` stages.**

In [15]:
sort_query = [
          { "$sort": { "imdb.rating": -1 } },
          { "$limit": 5 },
          { "$project": { "title": 1, "_id": 0} } ]

for doc in collection.aggregate(sort_query):
    print(doc)

{'title': 'The Danish Girl'}
{'title': 'Landet som icke èr'}
{'title': 'Scouts Guide to the Zombie Apocalypse'}
{'title': 'Catching the Sun'}
{'title': 'La nao capitana'}


### Top 5 directors by average IMDB rating, who have made at least 20 movies

Let's look at a slightly more complex aggregation pipeline. Here, we want to find the top 5 directors by average IMDB rating, who have made more than 20 movies. Once again, let's break down the calculation into sub-steps:
1. Extract individual directors from the array of `directors`.
2. For each director, count the number of films made, and the average `imdb.rating`.
3. Filter for directors who have made >= 20 movies.
4. Sort the resulting list of directors in descending ourder of average IMDB rating.
5. Limit the number of results returned to 5.

You learned about the `$sort` and `$limit` stages in the previous example. We will use these in Steps 4 and 5 of the pipeline, but you will need the following stages for Step 1 to 3:
1. [`$unwind`](https://www.mongodb.com/docs/manual/reference/operator/aggregation/unwind/): Deconstruct a single document with a list of directors into multiple documents, one for each director.
2. [`$group`](https://www.mongodb.com/docs/manual/reference/operator/aggregation/group/): Group documents by director, and calculate additional fields such as film count (`filmCount`) and average `imdb.rating` (`avgRating`)using accumulator expressions such as `$sum` and `$avg` respectively. Refer to MongoDB's documentation to see all the [accumulator expressions](https://www.mongodb.com/docs/manual/reference/operator/aggregation/group/#std-label-accumulators-group) supported by the MongoDB Query API.
3. [`$match`](https://www.mongodb.com/docs/manual/reference/operator/aggregation/match/): Filter directors based on condition, in this case, film count >= 20.

In [16]:
group_query = [ { "$unwind": "$directors" },
          { "$group": { "_id": "$directors", "filmCount": { "$sum": 1 }, "avgRating": { "$avg": "$imdb.rating" } } },
          { "$match": { "filmCount": { "$gte": 20 } } },
          { "$sort": { "avgRating": -1 } },
          { "$limit": 5 } ]

# Execute the aggregation pipeline and iterate through the resulting cursor
for doc in collection.aggregate(group_query):
    print(doc)

{'_id': 'William Wyler', 'filmCount': 21, 'avgRating': 7.676190476190476}
{'_id': 'Martin Scorsese', 'filmCount': 32, 'avgRating': 7.640625}
{'_id': 'Alfred Hitchcock', 'filmCount': 24, 'avgRating': 7.5874999999999995}
{'_id': 'Steven Spielberg', 'filmCount': 29, 'avgRating': 7.479310344827587}
{'_id': 'Woody Allen', 'filmCount': 40, 'avgRating': 7.215000000000001}
